## Step 1: Setup and Load Data

We'll use synthetic heating-time dataset where Raman spectra are measured at multiple time points.

In [ ]:
import matplotlib.pyplot as plt

from foodspec.apps.heating import run_heating_quality_workflow
from foodspec.demo import synthetic_heating_dataset
from foodspec.viz.heating import plot_ratio_vs_time

print("✓ All imports successful")

## Step 2: Load Synthetic Heating Dataset

The dataset contains spectra from multiple oils measured at successive heating time points.

In [ ]:
print("Loading synthetic heating dataset...")
fs = synthetic_heating_dataset()

print("\nDataset Summary:")
print(f"  Shape: {len(fs.data)} samples × {fs.data.x.shape[1]} wavenumbers")
print(f"  Modality: {fs.data.modality.upper()}")

# Examine time-series structure
heating_times = sorted(fs.data.metadata['heating_time'].unique())
print(f"\nTime Points Measured: {len(heating_times)} points")
print(f"  Range: {heating_times[0]:.1f} - {heating_times[-1]:.1f} hours")

oil_types = fs.data.metadata['oil_type'].unique()
print(f"\nOil Types: {oil_types}")
print(f"  Replicates per oil/time: {len(fs.data.metadata[fs.data.metadata['oil_type'] == oil_types[0]].loc[fs.data.metadata['heating_time'] == heating_times[0]])}")

## Step 3: Run Heating Quality Workflow

Extract key ratios (indicators of degradation) vs. heating time.

In [ ]:
print("Running heating quality workflow...")
result = run_heating_quality_workflow(fs, time_column="heating_time")

print("\n" + "="*60)
print("KEY RATIOS EXTRACTED")
print("="*60)
print(result.key_ratios.head(10))

print("\nRatio Columns (indicators of degradation):")
for col in result.key_ratios.columns:
    print(f"  - {col}")

## Step 4: Visualize Ratio Trends

Plot the first extracted ratio vs. heating time, with fitted trend model.

In [ ]:
# Get the first ratio
ratio_name = result.key_ratios.columns[0]
print(f"Analyzing ratio: {ratio_name}\n")

# Prepare data
heating_times = fs.data.metadata["heating_time"].values
ratio_values = result.key_ratios[ratio_name].values
trend_model = result.trend_models.get(ratio_name)

# Create figure
fig, ax = plt.subplots(figsize=(10, 6))
plot_ratio_vs_time(
    heating_times,
    ratio_values,
    model=trend_model,
    ax=ax,
)
ax.set_xlabel("Heating Time (hours)", fontsize=12)
ax.set_ylabel(f"{ratio_name}", fontsize=12)
ax.set_title(f"Oil Degradation Trend: {ratio_name}", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Interpretation:")
print("  - Upward trend = oxidation products accumulating")
print("  - Slope = degradation rate (steeper = faster oxidation)")
print("  - Critical threshold = oil freshness limit")

## Step 5: Estimate Shelf-Life

Use the trend model to predict when the ratio exceeds a critical threshold.

In [ ]:
# Extract model information
if trend_model is not None:
    if hasattr(trend_model, 'slope') and hasattr(trend_model, 'intercept'):
        slope = trend_model.slope
        intercept = trend_model.intercept

        print(f"\nTrend Model (Linear): y = {slope:.4f}*t + {intercept:.4f}")
        print(f"\nDegradation Rate: {slope:.4f} per hour")

        # Estimate critical threshold (e.g., 2x initial value)
        initial = intercept
        critical = 2 * initial
        time_to_critical = (critical - intercept) / slope if slope > 0 else float('inf')

        print(f"\nInitial ratio value: {initial:.4f}")
        print(f"Critical threshold (2x): {critical:.4f}")
        print(f"Estimated shelf-life: {time_to_critical:.1f} hours")
    else:
        print("\nModel available but coefficients not directly accessible.")
        print("Model type:", type(trend_model))
else:
    print("\nNo trend model fitted (insufficient samples or model error).")

## Step 6: Key Takeaways

### What We Learned:

1. **Time-Series Structure:** Spectroscopy data can track chemical changes over time
2. **Ratio Extraction:** Key ratios are more interpretable than raw spectra
3. **Trend Modeling:** Linear/nonlinear fits quantify degradation rates
4. **Shelf-Life Prediction:** From trend curves, we can estimate when products become unsafe
5. **Real-Time QC:** Monitor stored products by periodically measuring spectra

### Industrial Applications:

- **Batch Testing:** Screen incoming oils for oxidation status
- **Storage Monitoring:** Track product quality in warehouses
- **Predictive Maintenance:** Flag pallets approaching critical thresholds
- **Regulatory Reports:** Document stability test results for shelf-life claims

### Advantages of Spectroscopy for Stability:

- **Non-destructive:** No sample consumption (vs. chemical tests)
- **Fast:** Results in minutes (vs. days for standard tests)
- **Real-time:** Can monitor in-line during production
- **Quantitative:** Provides numerical degradation curves

### Next Steps:

- Try different critical thresholds (1.5x, 3x) and compare shelf-lives
- Compare trend models (linear, exponential, polynomial)
- Analyze multiple ratios to find the most stable indicator